In [25]:
import math
import itertools
import random
import time

###############################################################################
# 1) INCLUSION-EXCLUSION APPROACH
###############################################################################

def inclusion_exclusion_probability(N, K, r):
    """
    EXACT probability that no machine is chosen (r+1) times or more,
    using an inclusion-exclusion count. That is the probability that
    a conflict-free schedule in r rounds is possible.
    
    We compute:
        P = 1 - P( >=1 machine used >= r+1 times )
          = sum_{I subseteq [N]} (-1)^|I| * (# ways all machines in I appear >= r+1)
        and then divide by (C(N,r))^K (the total ways each of the K patrons can pick r out of N).
    
    WARNING: Runs in O(2^N) time, which is often not feasible if N > ~20 or 25.
    For very large N, you would generally prefer an approximation or a simulation.
    """
    from math import comb

    # Total number of ways (the denominator).
    total_ways = (comb(N, r))**K
    
    # We'll do standard inclusion-exclusion:
    #   ways_none = sum_{I subseteq [N]} [(-1)^{|I|} * # ways that each machine in I is chosen >= r+1 times]
    # and then Probability = ways_none / total_ways.
    
    ways_none = 0
    
    # Precompute binomial(N, k) for speed if needed, or just use math.comb directly.
    
    # We'll iterate over all subsets I of [N] by size for efficiency.
    # For each subset I, we need to count # ways that "for every j in I, machine j is used at least r+1 times".
    # Let m = |I|.
    # We do this by counting how to distribute the 'machine usage' among the K patrons, ensuring X_j >= r+1 for j in I.
    
    # A direct combinatorial approach:
    #   * Let X_j be the number of patrons who choose machine j.
    #   * Each patron's choice is an r-subset, so the total sum of X_j over j=1..N is r*K.
    #     Because each of the K patrons chooses exactly r machines, for a total of r*K machine-choices.
    #   * If we want all j in I to have X_j >= r+1, that means sum_{j in I} X_j >= (r+1)*m.
    #   * We also must ensure that each chosen set of X_j's is consistent with patrons actually forming r-subsets.

    # The simplest way for small K is "multi-variable" inclusion-exclusion again. We'll do a direct function:
    
    # Alternatively, a simpler trick: If ANY machine is chosen (r+1) times, that configuration is "bad".
    # So for subset I, counting "all machines in I have at least r+1" is not trivial. We'll define a separate function.
    
    # Implementation detail: We'll do a little recursive or combinatorial function that
    # enumerates ways for the K patrons to pick r-subsets s.t. *each j in I* is chosen >= (r+1) times.
    # Then we'll sum with the correct +/- sign.
    
    # For medium N, K, r, be aware this can still be expensive. We'll do a caching approach if possible.

    # We'll build an integer-based function:
    ways_cache = {}

    def ways_all_in_I_at_least_rplus1(I_tuple):
        """
        Count the number of ways for K patrons to pick r-subsets from N machines
        such that each machine in I_tuple appears at least (r+1) times total.
        We store results in ways_cache to avoid repeated calculations.
        """
        I_tuple = tuple(sorted(I_tuple))
        if not I_tuple:
            # If I is empty, there's no "at least r+1" constraint at all, so all ways are allowed:
            return total_ways
        if (I_tuple) in ways_cache:
            return ways_cache[I_tuple]
        
        m = len(I_tuple)
        
        # Approach: we must ensure that each j in I_tuple has X_j >= (r+1).
        # An alternative direct approach is to distribute at least (r+1) to each j in I_tuple,
        # then the remaining picks can be anything from the other machines, etc.
        # But we must be careful to ensure no patron picks > r machines.
        
        # We'll do a nested inclusion-exclusion approach:
        #   # ways( all j in I_tuple have X_j >= (r+1) )
        #   = sum_{empty != J subseteq I_tuple} (-1)^{|J|+...} # ways( each j in J has X_j >= (r+1) ), etc.
        # Actually, that is repeating the same pattern. Let's do a simpler logic:
        #
        # We can do it by direct "overcount - fix" method, but it's basically a second level of I-E.
        # For demonstration, let's do a naive enumerations of all (C(N, r))^K if K, N are small. That is too big if large.
        #
        # We'll implement a second-level inclusion-exclusion:
        
        # ways_any_j_in_J( X_j < r+1 ) for j in J is an easier "bad" event to handle, so we do:
        #
        # ways( all j in I have X_j >= r+1 )
        #   = sum_{S subseteq I} (-1)^{|S|} * ways( for j in S, X_j < r+1 ).
        
        # But we want X_j < r+1 to hold for *all j in S*, i.e. X_j <= r. Actually, that is the "good" event in the original sense.
        # We might invert it. Let's carefully do a standard formula:
        #
        #   ways( all j in I have X_j >= (r+1) )
        # = \sum_{S subseteq I} (-1)^{|S|}
        #       * ways( for j in S, X_j < (r+1) )  [the complement of "X_j >= r+1" for j in S].
        #
        # But that is again large. Let's adopt a direct or store approach. We'll do a simpler recursion for the # of solutions {which r-subsets are chosen} that satisfy X_j >= r+1 for each j in I_tuple.
        #
        # For clarity, let's do the *brute force* approach for demonstration, valid only for small K,N:
        
        count_valid = 0
        # Generate all ways the K patrons can pick r-subsets (extremely expensive if K,N not small!)
        from itertools import combinations, product
        all_r_subsets = list(combinations(range(N), r))  # all possible r-subsets from [0..N-1]
        
        for picks in product(all_r_subsets, repeat=K):
            # picks is a K-tuple of r-subsets
            # Build frequency count of how many times each machine is used
            freq = [0]*N
            for subset in picks:
                for mach in subset:
                    freq[mach] += 1
            # check if for each j in I_tuple, freq[j] >= r+1
            if all(freq[j] >= (r+1) for j in I_tuple):
                count_valid += 1
        
        ways_cache[I_tuple] = count_valid
        return count_valid

    # Now do the main inclusion-exclusion over subsets I of [N]
    #   ways_bad = # ways that there is at least one machine used >= (r+1) times
    # => ways_none = total_ways - ways_bad
    # => ways_none = sum_{I subseteq [N]} (-1)^{|I|} * ways_all_in_I_at_least_rplus1(I)
    
    # We'll sum over all subsets I using bitmasks if N is small:
    from itertools import chain, combinations
    
    all_indices = range(N)
    def all_subsets(seq):
        """Generate all subsets of the given iterable (as lists)."""
        # You could also use itertools.combinations in a nested loop
        # or itertools.chain.from_iterable.
        s = list(seq)
        for size in range(len(s)+1):
            for c in combinations(s, size):
                yield c
    
    ways_none = 0
    for I in all_subsets(all_indices):
        if not I:
            # empty subset => ways_all_in_I_at_least_rplus1(empty) = total_ways
            sign = 1
        else:
            sign = (-1)**(len(I))
        ways_none += sign * ways_all_in_I_at_least_rplus1(I)
    
    # Probability is ways_none / total_ways
    prob = ways_none / total_ways
    return prob


###############################################################################
# 2) BRUTE FORCE ENUMERATION (for VERY small N,K,r)
###############################################################################

def brute_force_probability(N, K, r):
    """
    EXACT probability by enumerating *every* possible way the K patrons
    choose their r-subsets, and checking whether any machine is used >= (r+1) times.
    
    Only feasible if (C(N,r))^K is not too large!
    """
    from math import comb
    from itertools import combinations, product
    
    # All possible r-subsets from [0..N-1].
    all_r_subsets = list(combinations(range(N), r))
    total_configurations = (len(all_r_subsets))**K
    
    count_no_conflict = 0
    
    for picks in product(all_r_subsets, repeat=K):
        freq = [0]*N
        for subset in picks:
            for mach in subset:
                freq[mach] += 1
        # Check if any freq[mach] >= r+1
        if all(f <= r for f in freq):
            count_no_conflict += 1
    
    probability = count_no_conflict / total_configurations
    return probability


###############################################################################
# 3) MONTE CARLO SIMULATION
###############################################################################

def simulate_probability(N, K, r, trials=10_000_00):
    """
    Estimate the probability by randomly sampling each patron's choice of r machines
    from N, checking if no machine is used r+1 or more times. Return the fraction
    that passes the check.
    
    For large K, N, or r, this is often the only quick approach to get an approximate
    probability.
    """
    from math import comb
    from random import sample
    
    count_no_conflict = 0
    
    for _ in range(trials):
        # For each trial, generate random picks
        freq = [0]*N
        for _patron in range(K):
            chosen = sample(range(N), r)  # pick r distinct machines at random
            for m in chosen:
                freq[m] += 1
        # Check if conflict
        if max(freq) <= r:
            count_no_conflict += 1
    
    return count_no_conflict / trials

###############################################################################
# 4) DYNAMIC PROGRAMMING
###############################################################################

def dp_probability(N, K, r):
    """
    EXACT probability that no machine is chosen (r+1) times or more
    when K patrons each independently pick an r-subset from N distinct machines.

    That is equivalent to saying every machine is used at most r times.
    
    Uses a dynamic programming approach over the distribution of usage counts:
      DP_k[i_0, i_1, ..., i_r] = number of ways to assign r-subsets
                                for the first k patrons so that
                                exactly i_j machines are used j times (for j=0..r).
    
    Complexity is roughly O(K * N^(r+1)) in the worst case, since for each state
    we iterate over ways to pick r machines from usage buckets.  For small r (e.g. 2,3,4)
    this can be dramatically faster than naive inclusion-exclusion for moderate/large N.
    
    Returns:
      A floating-point probability in [0, 1].
    """
    from math import comb
    from collections import defaultdict
    
    # Total ways each of the K patrons can pick r out of N: (comb(N,r))^K
    total_ways = comb(N, r)**K
    
    # Initialize DP for k=0 patrons:
    # dp_current[(i_0, i_1, ..., i_r)] = number of ways
    # At start, all N machines are used 0 times => (N,0,0,...,0)
    init_state = tuple([N] + [0]*r)  # i_0=N, i_1=0,..., i_r=0
    dp_current = defaultdict(int)
    dp_current[init_state] = 1
    
    # A helper function to enumerate all ways to distribute "r" picks among usage=0..(r-1)
    # i.e. x_0 + x_1 + ... + x_{r-1} = r, with 0 <= x_j <= i_j
    def generate_distributions(i_counts, r):
        """
        i_counts is [i_0, i_1, ..., i_{r-1}],
        We want all integer (x_0,...,x_{r-1}) with sum = r and x_j <= i_j.
        Yields (x_0,...,x_{r-1}).
        """
        # We'll do a simple recursive generator:
        def backtrack(idx, remain, partial):
            if idx == len(i_counts)-1:
                x = remain
                if x <= i_counts[idx]:
                    yield partial + [x]
                return
            limit = min(i_counts[idx], remain)
            for x in range(limit+1):
                yield from backtrack(idx+1, remain - x, partial + [x])
        
        yield from backtrack(0, r, [])
    
    # Iteratively build dp for k=1..K:
    for k in range(K):
        dp_next = defaultdict(int)
        for state, ways_so_far in dp_current.items():
            # state = (i_0, i_1, ..., i_r), sum of i_j = N
            i_list = list(state)
            
            # We cannot pick from i_r because those machines have usage=r already
            # and can't be used more. So only i_0..i_{r-1} are relevant for new picks.
            i_counts = i_list[0:r]  # i_0..i_{r-1}, ignoring the final i_r
            # i_list[r] is how many machines have usage=r.
            
            # For each valid distribution x_0.. x_{r-1} of the new r picks:
            for xdist in generate_distributions(i_counts, r):
                # Number of ways to choose those actual machines is
                # product_j comb(i_j, xdist[j])
                pick_ways = 1
                for j, xj in enumerate(xdist):
                    if xj > 0:
                        pick_ways *= comb(i_counts[j], xj)
                
                # Now construct the new usage distribution:
                # i_j -> i_j - x_j
                # i_{j+1} -> i_{j+1} + x_j
                new_i_list = i_list[:]  # copy
                for j, xj in enumerate(xdist):
                    new_i_list[j] -= xj      # those used-j machines become used-(j+1)
                    new_i_list[j+1] += xj    # increment usage in the next bucket
                # That defines the new state:
                new_state = tuple(new_i_list)
                
                dp_next[new_state] += ways_so_far * pick_ways
        
        # Move dp_next -> dp_current
        dp_current = dp_next
    
    # After K patrons, sum all dp_current[state] over all possible states
    # That sum is the number of ways for which no machine usage exceeded r.
    count_no_conflict = sum(dp_current.values())
    
    # Probability:
    prob = count_no_conflict / total_ways
    return prob


###############################################################################
# DEMO / EXAMPLE
###############################################################################
if __name__ == "__main__":
    # A small example you can tweak:
    N = 20   # machines
    K = 10   # patrons
    r = 3   # each patron picks 2 distinct machines
    TRIALS = 100_000  # for simulation

    print("===== SMALL EXAMPLE =====")
    print(f"N={N}, K={K}, r={r}")

    # # 1) Inclusion-Exclusion Probability (Exact, feasible if N is small)
    # start_time = time.time()
    # prob_incl = inclusion_exclusion_probability(N, K, r)
    # incl_time = time.time() - start_time
    # print(f"Inclusion-Exclusion Probability = {prob_incl:.6f} (computed in {incl_time:.6f} seconds)")

    # # 2) Brute-Force Probability (Exact, feasible if (C(N,r))^K is not too large)
    # start_time = time.time()
    # prob_brute = brute_force_probability(N, K, r)
    # brute_time = time.time() - start_time
    # print(f"Brute-Force Probability        = {prob_brute:.6f} (computed in {brute_time:.6f} seconds)")

    # 3) DP Probability (Exact, feasible if K is not too large)
    start_time = time.time()
    prob_dp = dp_probability(N, K, r)
    dp_time = time.time() - start_time
    print(f"DP Probability                 = {prob_dp:.6f} (computed in {dp_time:.6f} seconds)")

    # 4) Simulation
    start_time = time.time()
    prob_sim = simulate_probability(N, K, r, trials=TRIALS)
    sim_time = time.time() - start_time
    print(f"Simulation Probability         = {prob_sim:.6f} (computed in {sim_time:.6f} seconds, {TRIALS} trials)")


===== SMALL EXAMPLE =====
N=20, K=10, r=3
DP Probability                 = 0.294051 (computed in 0.021353 seconds)
Simulation Probability         = 0.294320 (computed in 3.309194 seconds, 100000 trials)
